In [22]:
import os
import sys
from pathlib import Path

from arcgis.features import GeoAccessor
from ba_data_paths import ba_data

sys.path.append('../../src')
from geoai_retail import proximity_local as proximity
from geoai_retail import utils

sys.path.append('../../scripts')
import data_sources

In [11]:
data = Path(os.path.abspath('../../data'))
interim_dir = data/'interim'
raw_dir = data/'raw'
raw_gdb = raw_dir/'raw.gdb'

closest_brand = interim_dir/'closest_store.csv'
closest_comp = interim_dir/'closest_competition.csv'
origin_demographics = interim_dir/'origin_enrich_all.csv'
inrix_trips = interim_dir/'cust_count_inrix.csv'

comp_fc = raw_gdb/'sea_ace_comp'
comp_name_fld = 'CONAME'

study_loc_id = 409083547

In [19]:
dest_brand_df = GeoAccessor.from_featureclass(data_sources.destination_store_locations)
dest_brand_df.drop('OBJECTID', axis=1, inplace=True)
dest_brand_df.head()

,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,...,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,NEAR_FID,NEAR_DIST,SHAPE
0,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,525104,...,,6,PointAddress,M,100.0000,INFOGROUP,0,3,0.000021,"{""x"": -122.29909999999995, ""y"": 47.05410000000..."
1,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,TACOMA,WA,Washington,98498,6154,44413005,525104,...,,3,StreetAddress,M,96.3594,INFOGROUP,0,35,0.000010,"{""x"": -122.54489999999998, ""y"": 47.18000000000..."
2,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,525104,...,,7,StreetAddress,M,100.0000,INFOGROUP,0,41,0.000019,"{""x"": -122.57929999999999, ""y"": 47.30130000000..."
3,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,525104,...,,5,PointAddress,M,100.0000,INFOGROUP,0,96,55.586255,"{""x"": -122.43459999999999, ""y"": 47.08740000000..."
4,721714069,ACE HARDWARE,112TH ST S,PARKLAND,WA,Washington,98444,5711,44413005,525104,...,,3,PointAddress,M,95.4063,INFOGROUP,0,98,0.000010,"{""x"": -122.43199999999996, ""y"": 47.15610000000..."


In [21]:
data_sources.origin_customer_areas

'D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\sea_block_group'

In [27]:
closest_sol_df = proximity.get_closest_solution(data_sources.origin_customer_areas, data_sources.origin_customer_area_id_field, 
                               data_sources.destination_store_locations, data_sources.destination_store_id_field,
                               network_dataset=ba_data.usa_network_dataset, destination_count=6)
closest_sol_df.head()

,origin_id,destination_rank,destination_id,proximity_traveltime,proximity_kilometers,SHAPE
0,530530701003,1,677129595,43.576384,38.513638,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
1,530530701003,2,371889957,46.482533,42.526899,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
2,530530701003,3,427271369,52.472263,44.462895,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
3,530530701003,4,421027779,55.156955,44.748810,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
4,530530701003,5,721714069,61.428125,50.456415,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."


In [28]:
closest_sol_df.spatial.to_featureclass(str(interim_dir/'interim.gdb'/'closest_solution'))

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\closest_solution'

In [39]:
origin_df = GeoAccessor.from_featureclass(str(data_sources.origin_customer_areas))
origin_df[data_sources.origin_customer_area_id_field] = origin_df[data_sources.origin_customer_area_id_field].astype(str)
origin_df.set_index(data_sources.origin_customer_area_id_field, inplace=True, drop=True)
origin_df.drop(['OBJECTID', 'NAME'], inplace=True, axis=1)
origin_df.head()

,SHAPE
ID,
530530701003,"{""rings"": [[[-122.06627999999995, 47.076520000..."
530530714071,"{""rings"": [[[-122.34031999999996, 47.071510000..."
530530714072,"{""rings"": [[[-122.35767999999996, 47.067370000..."
530530714073,"{""rings"": [[[-122.36847999999998, 47.067630000..."
530530714112,"{""rings"": [[[-122.41108999999994, 47.071690000..."


In [43]:
origin_closest_df = closest_sol_df.drop('SHAPE', axis=1).join(origin_df, on='origin_id')
origin_closest_df.head()

,origin_id,destination_rank,destination_id,proximity_traveltime,proximity_kilometers,SHAPE
0,530530701003,1,677129595,43.576384,38.513638,"{'rings': [[[-122.06627999999995, 47.076520000..."
1,530530701003,2,371889957,46.482533,42.526899,"{'rings': [[[-122.06627999999995, 47.076520000..."
2,530530701003,3,427271369,52.472263,44.462895,"{'rings': [[[-122.06627999999995, 47.076520000..."
3,530530701003,4,421027779,55.156955,44.748810,"{'rings': [[[-122.06627999999995, 47.076520000..."
4,530530701003,5,721714069,61.428125,50.456415,"{'rings': [[[-122.06627999999995, 47.076520000..."


In [44]:
origin_closest_df.spatial.to_featureclass(str(interim_dir/'interim.gdb'/'closest_origin_solution'))

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\closest_origin_solution'

In [46]:
closest_sol_comp_df = proximity.get_closest_solution(data_sources.origin_customer_areas, 
                                                     data_sources.origin_customer_area_id_field, 
                                                     data_sources.destination_competition_locations, 
                                                     data_sources.destination_competition_id_field,
                                                     network_dataset=ba_data.usa_network_dataset, 
                                                     destination_count=6)
closest_sol_comp_df.head()

,origin_id,destination_rank,destination_id,proximity_traveltime,proximity_kilometers,SHAPE
0,530530701003,1,388273260,27.324552,24.844065,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
1,530530701003,2,174262691,26.486351,25.466621,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
2,530530701003,3,679623777,30.496295,25.960217,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
3,530530701003,4,403218374,30.669763,26.656119,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."
4,530530701003,5,948351200,33.274677,31.095850,"{""hasM"": true, ""paths"": [[[-122.03589, 47.0116..."


In [47]:
closest_sol_comp_df.spatial.to_featureclass(str(interim_dir/'interim.gdb'/'closest_origin_competition_solution'))

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\closest_origin_competition_solution'